In [1]:
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode
import calendar
import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import xlsxwriter
from tzlocal import get_localzone
import dateutil
import pandas_gbq
from google.oauth2 import service_account
import config

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
class Spotify_UserInput:
    user_time_range = {"1":"short_term", "2":"medium_term", "3":"long_term"}

    time_range_reminder = ["1. short_term = approximately last 4 weeks", "2. medium_term = approximately last 6 months",
                          "3. long_term = calculated from several years of data and including all new data as it becomes available"]
    
    def user_timerange(user_time_range, time_range_reminder):
        '''
        Ask user for specify time range
        - short_term = approximately last 4 weeks
        - medium_term = approximately last 6 months
        - long_term = calculated from several years of data and including all new data as it becomes available
        '''
        while True:
            print("How far back would you like to go?"
                  "\nSelect time range by typing the number 1-3, your choices: "
                  "\n1. short_term = approximately last 4 weeks"
                  "\n2. medium_term = approximately last 6 months"
                  "\n3. long_term = calculated from several years of data and including all new data as it becomes available"
                  "\nYour input: ")
            date_range = input("Time range #: ")
            i = 0

            while date_range not in user_time_range.keys():
                date_range = input("Invalid entry. Please select time range with number above: ")
                i += 1
                if i == 2:
                    print("\nLooks like you're having trouble. Here are the available inputs: \n")
                    print(time_range_reminder)
                    print()
                    date_range = input("Enter a court number: ")
                    i += 1
                if i >= 3 and i % 3:
                    quitting = input("Do you want to quit (yes/no)? ").lower()
                    if quitting == "yes":
                        break
                    else:
                        date_range = input("Select time range by typing the number 1-3, your choices: "
                                  "\n1. short_term = approximately last 4 weeks"
                                  "\n2. medium_term = approximately last 6 months"
                                  "\n3. long_term = calculated from several years of data and including all new data as it becomes available"
                                  "\nYour input: ")
                if date_range not in user_time_range.keys():
                    quitting = "yes"
                    return quitting
            print(f"Pulling '{user_time_range[date_range]}' data...")
            return user_time_range[date_range]

    
class Spotify_Auth:
    
    def __init__(self, client_id=None, client_secret=None, username=None):
        self.client_id = client_id
        self.client_secret = client_secret
        self.username= username
        
    
    def authenticate(self):
        redirect_uri = 'http://localhost:7777/callback'
        scope = 'user-read-recently-played, user-top-read'

        auth_token = util.prompt_for_user_token(username=self.username, 
                                          scope=scope, 
                                          client_id=self.client_id,   
                                          client_secret=self.client_secret,     
                                          redirect_uri=redirect_uri)
        return auth_token

    
class Spotify_Data:
    
    def recently_played(auth_token):
        base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
        #track_id = '6y0igZArWVi6Iz0rj35c1Y'

        #2. Authentication
        #3. Parameters -- would be stored with authentication
        headers = {
            "Authorization": f"Bearer {auth_token}"
        }

        #4. Create an empty list
        personal_data = [] #would be good explore how to capture data at different points in time
        r = requests.get(base_url+"&limit=50", headers=headers)
        personal_data.append(json.loads(r.text))
        
        return personal_data
    
    def top_tracks(auth_token, time_range):
        
        base_url = 'https://api.spotify.com/v1/me/top/tracks?'
        #track_id = '6y0igZArWVi6Iz0rj35c1Y'

        #2. Authentication
        #3. Parameters -- would be stored with authentication
        headers = {
            "Authorization": f"Bearer {auth_token}"
        }

        #4. Create an empty list
        top_track_data = [] #would be good explore how to capture data at different points in time
        r = requests.get(base_url+f"time_range={time_range}"+"&&limit=50", headers=headers)
        top_track_data.append(json.loads(r.text))
        
        return top_track_data
        

        
class Data_Clean:
    
    def recently_played_clean(personal_data):
        track_ids = []
        album_ids = []
        artist_ids = []
        album_names = []
        artist_names = []
        track_names = []
        popularity_ls = []
        played_ats = []

        for i in range(len(personal_data[0]['items'])):
            track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
            album_ids.append(personal_data[0]['items'][i]['track']['album']['id']) # Albumn ID
            artist_ids.append(personal_data[0]['items'][i]['track']['artists'][0]['id']) # Artist ID
            album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
            artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
            track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name
            popularity_ls.append(personal_data[0]['items'][i]['track']['popularity']) # Track Popularity
            played_ats.append(personal_data[0]['items'][i]['played_at'])

        list_dic={'track_id':track_ids,
              'album_id':album_ids,
              'artist_id':artist_ids,
              'track_name':track_names,
              'artist_name':artist_names,
              'album_name':album_names,
              'track_popularity':popularity_ls,
              'time_palyed': played_ats
        }

        df1=pd.DataFrame(list_dic)

        # Create empty column to append data to
        df1['local_time'] = ''

        for i in range(len(personal_data[0]['items'])):
            # Convert UTC to local time zone
            utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[i]).astimezone(get_localzone())
            # Format date/time
            local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
            local_time

            df1['local_time'].iloc[i] = local_time

        return df1
    
        def top_tracks_clean(top_track_data):
        track_idss = []
        album_idss = []
        artist_idss = []
        album_namess = []
        album_relase_datess = []
        artist_namess = []
        popularity_lss = []
        track_namess = []

        for i in range(len(top_track_data[0]['items'])):
            track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
            album_idss.append(top_track_data[0]['items'][i]['album']['id']) # Album ID
            artist_idss.append(top_track_data[0]['items'][i]['album']['artists'][0]['id'])
            album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
            album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
            artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
            popularity_lss.append(top_track_data[0]['items'][i]['popularity'])
            track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name
            
        list_dic2 = {'track_id':track_idss,
           'album_id':album_idss,
           'artist_id':artist_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'track_popularity':popularity_lss,
           'album_relase_date':album_relase_datess,
                  }
        
        df2=pd.DataFrame(list_dic2)
        
        return df2


In [4]:
token = Spotify_Auth(config.client_id, config.client_secret, config.username)
token

In [5]:
token.client_id

'7b48c0d0afaf473fa001e013a4da575b'

In [7]:
token.username

'1221014369'

In [8]:
token.authenticate()

'BQBRGAraqxD05108SNcuwNGoYhvDlVgbug7eX-zuXWFy9QdGpLuXhVymbPV82HaSu7ZB0yWiy8YPK3vTseq5VfkbWvTLncZNbuHFIjIR1Q2lteaqwFTpd0gbZLY7VOYXDWxmBrkh6vT60R6gsVk88gPyUZZ3D6mqgpnDOsiJ0OIkx0zSHJIZKsZvOa07IU_5IhDGQUloZRpjs_7SL6w24L6ONmzi8RQA28W1'

In [9]:
Data_Clean.recently_played_clean(Spotify_Data.recently_played(token.authenticate()))

,track_id,album_id,artist_id,track_name,artist_name,album_name,track_popularity,time_palyed,local_time
0,7MypU6ZEqL6hstZYykaraL,7F6XbqxHV4xIAj14oUCZoS,0Raaw7kr1Vzat4ZvHzjsJR,Oh Noel,I DONT KNOW HOW BUT THEY FOUND ME,Christmas Drag,41,2020-11-25T01:21:01.919Z,2020-11-24 19:21:01
1,0TuoTBdYsepQ2VKrWDW5hM,7F6XbqxHV4xIAj14oUCZoS,0Raaw7kr1Vzat4ZvHzjsJR,Merry Christmas Everybody,I DONT KNOW HOW BUT THEY FOUND ME,Christmas Drag,41,2020-11-25T01:14:25.648Z,2020-11-24 19:14:25
2,61EROL9kSJ5qhn1UH9mqc5,7F6XbqxHV4xIAj14oUCZoS,0Raaw7kr1Vzat4ZvHzjsJR,Christmas Drag,I DONT KNOW HOW BUT THEY FOUND ME,Christmas Drag,40,2020-11-25T01:11:00.144Z,2020-11-24 19:11:00
3,1b0FdeyNJ0LuQ4ha8API9G,0JEbkW1a2Ws7P8vMflSPc9,69tiO1fG8VWduDl3ji2qhI,Every Holiday,Mt. Joy,Rearrange Us,44,2020-11-25T01:06:44.401Z,2020-11-24 19:06:44
4,3e3d7FRtLL43hvnqUeJAwq,31i8FCqq0rEND2VDPzgkZY,67WNUxmM7y4WzHPAVzBu3E,Holiday-ish (feat. Dylan Minnette),The Regrettes,Holiday-ish (feat. Dylan Minnette),39,2020-11-25T01:03:03.148Z,2020-11-24 19:03:03
5,3GhsBdS9ulPK3KCdwHRPhG,6w9osZGOS8B7XWW51SBwFT,1QAJqy2dA3ihHBFIHRphZj,Nothing's Gonna Hurt You Baby,Cigarettes After Sex,I.,63,2020-11-24T22:16:45.428Z,2020-11-24 16:16:45
6,7LX4aXtvImhdgUHQBlo8Kp,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,K.,Cigarettes After Sex,Cigarettes After Sex,61,2020-11-24T22:13:04.085Z,2020-11-24 16:13:04
7,3AVrVz5rK8Hrqo9YGiVGN5,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,Apocalypse,Cigarettes After Sex,Cigarettes After Sex,70,2020-11-24T22:07:17.612Z,2020-11-24 16:07:17
8,7LX4aXtvImhdgUHQBlo8Kp,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,K.,Cigarettes After Sex,Cigarettes After Sex,61,2020-11-24T01:38:22.975Z,2020-11-23 19:38:22
9,3AVrVz5rK8Hrqo9YGiVGN5,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,Apocalypse,Cigarettes After Sex,Cigarettes After Sex,70,2020-11-24T01:35:39.626Z,2020-11-23 19:35:39


In [10]:
Data_Clean.top_tracks_clean(Spotify_Data.top_tracks(token.authenticate(), Spotify_UserInput.user_timerange(Spotify_UserInput.user_time_range, Spotify_UserInput.time_range_reminder)))

How far back would you like to go?
Select time range by typing the number 1-3, your choices: 
1. short_term = approximately last 4 weeks
2. medium_term = approximately last 6 months
3. long_term = calculated from several years of data and including all new data as it becomes available
Your input: 
Time range #: 3
Pulling 'long_term' data...


,track_id,album_id,artist_id,track_name,album_name,artist_name,track_popularity,album_relase_date
0,4w2tfK0JA8KrVegKnxukf4,4XOMJHVuzJVWmqUdp4SYKP,5tFRohaO5yEsuJxmMnlCO9,The Kids Are Alright,404,Barns Courtney,36,2019-09-06
1,3vTG7B4eu8jsrSQGuxYygG,7wRp9uZPJVTPnDWECDAyuu,6lcBiGiT3dlyDMjBBtfyfS,A Toast and a Spirit,A Toast and a Spirit,Vacation Manor,0,2015-10-16
2,70pYds3cGjJv2hnPie9c0H,2EAuRn2cglaAseq0G3ntN8,7MPGCB854Qo4alYMOPkBka,Young Adult,Young Adult,Ritt Momney,55,2017-02-22
3,3rSkiyzwE3oQXhvMqobU0e,6blMxezujKgPe8HjHNveuG,4j56EQDQu5XnL7R3E9iFJT,You + I,Mirror Master,Young the Giant,41,2018-10-12
4,42SOtYBdnfj9zEdZSlAq5k,68enXe5XcJdciSDAZr0Alr,26VFTg2z8YR0cCuwLzESi2,Finally // beautiful stranger,Manic,Halsey,62,2020-01-17
5,7DPQwyMQADl9Y8oSvSVpfg,0leVOEw8L2TnAfLGwkFnab,142YBUGmLWCJigFLzgguf8,Let Me Down Easy,Go Farther In Lightness,Gang of Youths,63,2017-08-18
6,00c805RUf4tZyrlycaA2AQ,4XOMJHVuzJVWmqUdp4SYKP,5tFRohaO5yEsuJxmMnlCO9,London Girls,404,Barns Courtney,44,2019-09-06
7,4pvb0WLRcMtbPGmtejJJ6y,2fenSS68JI1h4Fo296JfGr,06HL4z0CvFAxyc27GXpf02,exile (feat. Bon Iver),folklore,Taylor Swift,82,2020-07-24
8,0dgXDPDvQNPKUYNR3KZeHS,0leVOEw8L2TnAfLGwkFnab,142YBUGmLWCJigFLzgguf8,What Can I Do If the Fire Goes Out?,Go Farther In Lightness,Gang of Youths,53,2017-08-18
9,4c8VzUIySeAHdASD0VWHPo,05zUfCpCWiK4ak7YH5ohmS,7MPGCB854Qo4alYMOPkBka,Pollution / Disclaimer,Pollution / Disclaimer,Ritt Momney,0,2018-09-06
